In [1]:
import os
import sys
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 5 --executor-memory 5g pyspark-shell'
#os.environ["PYSPARK_SUBMIT_ARGS"]='pyspark-shell'
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'

spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')
sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.4.7
      /_/

Using Python version 3.6.5 (default, Apr 29 2018 16:14:56)
SparkSession available as 'spark'.


In [2]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql.types import ArrayType, DoubleType, BooleanType, FloatType
import pyspark.sql.functions as f
from pyspark.sql.functions import col, array_contains, split, array_distinct, explode, collect_set, when, lit
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, Normalizer, StopWordsRemover
import pandas as pd
import numpy as np

Данные:
- TRAIN - датасет, на котром тренируется модель
- TEST - датасет, на котром проверям качество своей модели, целевой признак отсутствует (без указания purchase, который и нужно предсказать)
- ITEMS - весь контент, который представлен на платформе E-Contenta (много ненужной информации, поля нужно фильтровать). Здесь представлена подробная информация о контенте, который предложен пользователям. Нам нужны тольео 5 полей. Причем в поле жанр может быть несколько жанров, которые перечислены через запятую => на это нужно обратить внимание.
- VIEW PROGRAMMES - метрики и дополнительные данные о просмотрах пользователями. В частности здесь указано, как долго пользовательс мотрел ту или иную идиницу контента. 

In [3]:
schema_train = StructType(fields = [
    StructField("user_id", IntegerType(), True),
    StructField("item_id", IntegerType(), True),
    StructField("purchase", IntegerType(), True),
])

df_train = spark.read.csv("/labs/slaba03/laba03_train.csv", sep = ",", header = True, schema = schema_train)
df_train.show(3,  vertical = False, truncate = False)

+-------+-------+--------+
|user_id|item_id|purchase|
+-------+-------+--------+
|1654   |74107  |0       |
|1654   |89249  |0       |
|1654   |99982  |0       |
+-------+-------+--------+
only showing top 3 rows



In [4]:
schema_test = StructType(fields = [
    StructField("user_id", IntegerType(), True),
    StructField("item_id", IntegerType(), True)
])

test = spark.read.csv("/labs/slaba03/laba03_test.csv", sep = ",", header = True, schema = schema_test)
test.show(3, vertical = False, truncate = False)

+-------+-------+
|user_id|item_id|
+-------+-------+
|1654   |94814  |
|1654   |93629  |
|1654   |9980   |
+-------+-------+
only showing top 3 rows



In [5]:
schema_items = StructType(fields = [
    StructField("item_id", IntegerType()),
    StructField("channel_id", IntegerType()),
    StructField("datetime_availability_start", StringType()),
    StructField("datetime_availability_stop", StringType()),
    StructField("datetime_show_start", StringType()),
    StructField("datetime_show_stop", StringType()),
    StructField("content_type", IntegerType()),
    StructField("title", StringType(), nullable = True),
    StructField("year", FloatType(), nullable = True),
    StructField("genres", StringType()),
    StructField("region_id", IntegerType())
])

df_items = spark.read.csv("/labs/slaba03/laba03_items.csv", sep = "\t", header = True, schema = schema_items)
df_items = df_items.select(["item_id", "content_type", "title", "year", "genres"])
df_items.show(2,  vertical = True, truncate = False)

-RECORD 0----------------------------------------------------------------
 item_id      | 65667                                                    
 content_type | 1                                                        
 title        | на пробах только девушки (all girl auditions)            
 year         | 2013.0                                                   
 genres       | Эротика                                                  
-RECORD 1----------------------------------------------------------------
 item_id      | 65669                                                    
 content_type | 1                                                        
 title        | скуби ду: эротическая пародия (scooby doo: a xxx parody) 
 year         | 2011.0                                                   
 genres       | Эротика                                                  
only showing top 2 rows



In [6]:
df_items = df_items.na.fill(value=-1,subset=["year"])

In [7]:
df_items.select('content_type').distinct().collect()

[Row(content_type=1), Row(content_type=0)]

In [8]:
schema_programmes = StructType(fields = [
    StructField("user_id", IntegerType()),
    StructField("item_id", StringType()),
    StructField("ts_start", IntegerType()),
    StructField("ts_end", IntegerType()),
    StructField("ite_type", StringType())
])

df_programmes = spark.read.csv("/labs/slaba03/laba03_views_programmes.csv", sep = ",", header = True, 
                              schema = schema_programmes)
df_programmes.show(3)

+-------+-------+----------+----------+--------+
|user_id|item_id|  ts_start|    ts_end|ite_type|
+-------+-------+----------+----------+--------+
|      0|7101053|1491409931|1491411600|    live|
|      0|7101054|1491412481|1491451571|    live|
|      0|7101054|1491411640|1491412481|    live|
+-------+-------+----------+----------+--------+
only showing top 3 rows



### План решения
1. **Генерация пизнакови и фильтрация**. Нужно составитьтакие признаки, которые охарактеризовали бы покупаемость того или иного контента с некоторой вероятностью. Также надо составить статистику для users, с какой вероятностью этот user вообще покупает како-либо контент. 
   - 2 типа признаков: 1) характеризуют покупаемость того или иного контента (то есть с какой веротяностью покупается контент в зависимости от признаков этого контента => для этого используем one hot encoding), 2) user-statistics, с какой вероятностью вообще покупается какой то контент (для этого используем target encoding).
   - В таблице ITEMS есть бинарный признак content_type - нас интересуют только платные передачи. Поэтому необходимо сделать фильтрацию. 
   - В таблице ITEMS нас интересуют только пять полей: item_id, content_type, title, year, genres.
   - **Target Encoding** - это encoding для категориальных признаков, который содержит в себе инофрмацию о частоте таргета в категориальных признаках. То есть с какой веротяностью таргет (единица) встречается в том или ином категориальном признаке. Перед использованием Target Encoding необходимо разделить тренировочный датасет на 3 части - train, validation и датасет для расчета статистики. Мы считатем статситсику по юзерам и контенту на раннем промежутке времени, а дальше сделать join этой статситики к train и validation и test. Это показательный признак, который попожет знам в значительной степени. 
       - Например, мы можем посчитать статситику, нкакова вероятность, что конкретный user/фильм вообще покупаются (на основе laba03_train посчитать, как интенсивно покупает пользователь и покупаемость items)
   - Если в признаке есть несколько жанров, то они перечислены через запятую => на это нужно обратить внимание. Для обработки жанров будет использоваться **One Hot Encoding** (!!! НО при этом не нужно использовать onehotencoding для всех комбинаций жанров, так как этих комбинаций жанров очень много => это очень сильно раздует объем фичей). Также ожно применить CountVectorizer к жанрам и полученный вектор  использовать как фичу. 
   - Необходимо использовать информацию по времени просмотра, которая представлена в файле VIEW PROGRAMME. Таймстэмпы ts_start и ts_end можно использовать для feature-engineering. Можно рассчитать время простмотра для разных типов просмотров - live и pvr.
2. **Построение модели GBTClassifier**

In [9]:
# Разбиваем на train и validation
train = df_train.sampleBy("purchase", fractions={0: 0.8, 1: 0.8}, seed=5757)

valid = df_train.join(train, on=["user_id", "item_id"], how="leftanti")


In [10]:
train_prob = train.groupBy('user_id')\
                .sum().select(col("sum(purchase)").alias("user_purchases"), col("user_id")).cache()


In [11]:
item_prob = train.groupBy('item_id')\
                        .sum().select(col("sum(purchase)").alias("item_prob"), col("item_id")).cache()

In [12]:
train = train.join(train_prob, on='user_id', how='left')
valid = valid.join(train_prob, on='user_id', how='left')
test = test.join(train_prob, on='user_id', how='left')

train = train.join(item_prob, on='item_id', how='left')
valid = valid.join(item_prob, on='item_id', how='left')
test = test.join(item_prob, on='item_id', how='left')

In [13]:
train_user_nums = train.groupBy('user_id').count().select(col("count").alias("user_attempts"), col("user_id"))\
                            .cache()

train_item_attempts = train.groupBy('item_id').count().select(col("count").alias("item_attempts"), col("item_id"))\
                            .cache()

train_user_nums.show()

+-------------+-------+
|user_attempts|user_id|
+-------------+-------+
|         2089| 754230|
|         2058| 761341|
|         2059| 780033|
|         2089| 798454|
|         2104| 825061|
|         2032| 833685|
|         2114| 846231|
|         2128| 851486|
|         2089| 867850|
|         2122| 870928|
|         2064| 927211|
|         2084| 776188|
|         2103| 901457|
|         2045| 879401|
|         2064| 928140|
|         2080| 824008|
|         2073| 880451|
|         2063| 890476|
|         2070| 900203|
|         2094| 937345|
+-------------+-------+
only showing top 20 rows



In [14]:
train = train.join(train_user_nums, on='user_id', how='left')
valid = valid.join(train_user_nums, on='user_id', how='left')
test = test.join(train_user_nums, on='user_id', how='left')

train = train.join(train_item_attempts, on='item_id', how='left')
valid = valid.join(train_item_attempts, on='item_id', how='left')
test = test.join(train_item_attempts, on='item_id', how='left')

In [15]:
train = train.withColumn('user_addict', (train.user_purchases / train.user_attempts))
valid = valid.withColumn('user_addict', col('user_purchases') / col('user_attempts'))
test = test.withColumn('user_addict', col('user_purchases') / col('user_attempts'))

In [16]:
train = train.withColumn('item_addict', col('item_prob') / col('item_attempts'))
valid = valid.withColumn('item_addict', col('item_prob') / col('item_attempts'))
test = test.withColumn('item_addict', col('item_prob') / col('item_attempts'))

In [17]:
train_prob.unpersist()
item_prob.unpersist()
train_user_nums.unpersist()
train_item_attempts.unpersist()

DataFrame[item_attempts: bigint, item_id: int]

In [18]:
from pyspark.ml.feature import VectorAssembler
# Выбираю колонки, которые войдут в features для GBT
cols = ['item_prob', 'user_purchases', 'user_addict', 'item_addict']
assembler = VectorAssembler(inputCols=cols, outputCol="features")

train_data = assembler.transform(train).cache()
valid_data = assembler.transform(valid)
test_data = assembler.transform(test)

In [19]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import GBTClassifier

gbt = GBTClassifier(labelCol="purchase")

pipeline = Pipeline(stages=[
    gbt
])

In [20]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

evaluator = BinaryClassificationEvaluator(labelCol="purchase", metricName='areaUnderROC')


In [21]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

In [24]:
from pyspark.ml.classification import GBTClassifier

gbt = GBTClassifier(labelCol="purchase", maxDepth = 5, maxIter=2)

gbt_model = gbt.fit(train_data)
predictions_valid = gbt_model.transform(valid_data)

In [25]:
evaluator = BinaryClassificationEvaluator(labelCol="purchase", metricName='areaUnderROC')
score = evaluator.evaluate(predictions_valid)
score

0.7920015019857043

In [26]:
test_predictions = gbt_model.transform(test_data)

In [27]:
predictions_pd = test_predictions.select("user_id", "item_id", col("probability").alias("purchase")).toPandas()
predictions_pd = predictions_pd.sort_values(by=['user_id', 'item_id'])
predictions_pd['purchase'] = predictions_pd['purchase'].apply(lambda x: x[1])
predictions_pd.to_csv('lab03.csv', index=False)

In [28]:
spark.stop()